In [4]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import SimpleITK as sitk
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import seaborn as sns

INSTANCES_PATH = Path('/home/joasiee/Documents/instances/synthetic/scans/')

In [18]:
def get_img_data(path: Path):
    image = sitk.ReadImage(str(path.resolve()))
    return sitk.GetArrayFromImage(image)

def plot_voxels(data, y_slice_depth = 15, orientation=(0, -70), cmap_name='gray'):
    ax = plt.figure(figsize=(8,8)).add_subplot(projection='3d')
    sliced_data = np.copy(data)
    sliced_data[:, :y_slice_depth, :] = 0
    
    cmap = cm.get_cmap(cmap_name)
    norm = Normalize(vmin=np.min(sliced_data), vmax=np.max(sliced_data))

    colors = np.empty(sliced_data.shape, dtype=object)
    colors = np.array(list(map(lambda x: cmap(norm(x)), sliced_data)))

    ax.voxels(sliced_data, facecolors=colors, edgecolor='k')
    ax.set_xlim3d(2, 28)
    ax.set_ylim3d(5, 28)
    ax.set_zlim3d(2, 28)
    ax.set_box_aspect((np.ptp(ax.get_xlim()), np.ptp(ax.get_ylim()), np.ptp(ax.get_zlim())))
    plt.locator_params(axis='y', nbins=3)
    ax.view_init(*orientation)

def plot_dvf(data, slice=15):
    X, Y = np.indices(data.shape[:2])
    u = data[:, slice, :, 0]
    v = data[:, slice, :, 2]
    colors = np.linalg.norm(data[:, slice, :, [0, 2]], axis=0) # np swaps smallest dim axis to 0
    
    fig, ax = plt.subplots(figsize =(8, 8))
    ax.quiver(X, Y, u, v, colors, angles='xy', scale_units='xy', cmap="Greys")
    plt.show()